In [1]:
# import libraries
import torch
from pathlib import Path

from neuralhydrology.nh_run import start_run, eval_run
import pickle
import matplotlib.pyplot as plt

### Train model with negative logarithm of the likelihood

In [2]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path("1_basin.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("1_basin.yml"), gpu=-1)

ValueError: ['hidden_layers'] are not recognized config keys.

### Evaluate run on test set
The path to the best model is automatically saved in the variable best_model, and the corresponding best epoch is stored as a string in best_epoch.

In [ ]:
eval_run(run_dir=best_model, period="test", epoch=best_epoch)

### Load and inspect model predictions
Load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [ ]:
with open(best_model / "test" / f"model_epoch0{best_epoch}" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

In [ ]:
results['DE4']['1D']['xr']

In [ ]:
# Extract observations and simulations
qobs = results['DE4']['1D']['xr']['discharge_vol_obs']
qsim = results['DE4']['1D']['xr']['discharge_vol_sim']

fig, ax = plt.subplots(figsize=(16,10))
ax.plot(qobs['date'], qobs)
ax.plot(qsim['date'], qsim)
ax.set_ylabel("Discharge (m³/s)")
ax.set_title(f"Test period - NSE {results['DE4']['1D']['NSE']:.3f}")